■ Patial Shot 구분하기

# 202041010
1. 각 shot의 4개모서리 좌표로 150mm를 넘어가는지 확인
2. 제품마다 step pitch가 다르기때문에 unique_id로 그룹지어서 계산


In [ ]:
# DF에 끝부분에 PARITAL 유무만 저장

In [2]:
import pandas as pd
import numpy as np


# 데이터 불러오기
file_path = 'RawData-1.csv'
df = pd.read_csv(file_path)


# Wafer 반경 설정
wafer_radius = 150000

# 각 unique_id 그룹별로 partial 여부 확인
partial_shot_info = []

# 그룹화하여 step pitch 값을 사용
for unique_id, group in df.groupby('UNIQUE_ID'):
    # 그룹 내에서 동일한 step pitch 값 사용
    step_pitch_x_value = group['STEP_PITCH_X'].iloc[0]
    step_pitch_y_value = group['STEP_PITCH_Y'].iloc[0]
    
    for _, row in group.iterrows():
        # 각 shot의 좌표
        dx, dy = row['DieX'], row['DieY']
        x, y = row['fcp_x'], row['fcp_y']
        
        # 각 모서리 좌표 계산
        bottom_left = (x - step_pitch_x_value / 2, y - step_pitch_y_value / 2)
        bottom_right = (x + step_pitch_x_value / 2, y - step_pitch_y_value / 2)
        top_left = (x - step_pitch_x_value / 2, y + step_pitch_y_value / 2)
        top_right = (x + step_pitch_x_value / 2, y + step_pitch_y_value / 2)
        
        # 각 모서리 좌표에서 원점으로부터의 거리 계산
        corners = [bottom_left, bottom_right, top_left, top_right]
        distances = [np.sqrt(c[0]**2 + c[1]**2) for c in corners]
        
        # 어떤 모서리라도 Wafer 반경을 초과하면 partial shot으로 구분
        is_partial = any(dist > wafer_radius for dist in distances)
        
        # 결과 저장
        shot_info = {
            "Unique_ID": unique_id,
            "Die_X": dx, 
            "Die_Y": dy,
            "Is Partial Shot": is_partial,
            "Bottom Left": bottom_left,
            "Bottom Right": bottom_right,
            "Top Left": top_left,
            "Top Right": top_right
        }
        
        partial_shot_info.append(shot_info)


# partial_shot_info 리스트를 DataFrame으로 변환
partial_shot_df = pd.DataFrame(partial_shot_info)

# 기존 DataFrame과 Is Partial Shot 열만 추출한 DataFrame을 우측 병합
df_patial = pd.concat([df, partial_shot_df['Is Partial Shot']], axis=1)

# 결과 확인


# DataFrame을 CSV 파일로 저장하기
output_file_path = 'RawData-1_is_partial_shot.csv'  # 저장할 파일 경로와 이름
df_patial.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"DataFrame이 '{output_file_path}' 파일로 저장되었습니다.")






DataFrame이 'RawData-1_is_partial_shot.csv' 파일로 저장되었습니다.
